In [1]:
#pip install
#%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core

In [1]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
# import fnmatch
# import mimetypes
# from neo4j import  Driver

# from dotenv import load_dotenv
# load_dotenv()

In [2]:
#PDF Loader

#path of the data directory
 


directory_path = os.getcwd() + "/documents"

all_pages = []

for root, dirs, files in os.walk(directory_path):
    for file in files:
        loader = PyPDFLoader(os.path.join(root, file))
        async for page in loader.alazy_load():
            all_pages.append(page)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=24)
documents = text_splitter.split_documents(documents=all_pages)
print(len(documents))

377


In [ ]:
# loader = TextLoader(file_path="dummytext.txt")
# docs = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=24)
# documents = text_splitter.split_documents(documents=docs)

# print(len(documents))

In [ ]:
for doc in documents:
    print(doc.metadata)    
    print(doc.page_content)
    print("----")


### Select LLM

In [4]:
#OpenAI chat  ***********************************
os.environ["OPENAI_API_KEY"] = "sk-proj-YOUR OWN KEY"
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


# # Ollama chat  ***********************************
# from langchain_ollama import ChatOllama
# llm = ChatOllama(model="llama3.1:8b-instruct-q4_0", temperature=0, format="json")

#Ollama OllamaFunctions ***********************************
# from langchain_experimental.llms.ollama_functions import OllamaFunctions
# llm = OllamaFunctions(model="llama3.1:8b-instruct-q4_0", temperature=0, format="json")

#GROQ Chat ***********************************
# from langchain_groq import ChatGroq
# os.environ["GROQ_API_KEY"] = 'gsk_YOUR OWN KEY'
# llm = ChatGroq( model="llama-3.1-8b-instant", verbose=True, temperature=0, max_tokens=None,timeout=None, max_retries=2,)
#llm = ChatGroq( model="llama-3.1-70b-versatile", verbose=True, temperature=0, max_tokens=None,timeout=None, max_retries=2,)
 
#vLLM Chat ***********************************
# from langchain_community.llms import VLLMOpenAI
# llm = VLLMOpenAI(
#     openai_api_key="EMPTY",
#     openai_api_base="http://localhost:8000/v1",
#     #model_name="microsoft/Phi-3-medium-128k-instruct",
#     model_name="neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w4a16",
#     #model_kwargs={"stop": ["."]},
#     temperature=0,)
# #Sanity Check for vLLM instance
# print(llm.invoke("What is the largest manmade planet in the solar system?"))


In [5]:

llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)


In [6]:
# Print all the nodes and relationships
for node in graph_documents[0].nodes:
    print(node)

print("-----")

for relationship in graph_documents[0].relationships:
    print(relationship) 

id='Tech Mahindra Limited' type='Company' properties={}
id='Sharda Centre, Off Karve Road, Pune - 411004, Maharashtra, India' type='Address' properties={}
id='+91 20 6601 8100' type='Phone' properties={}
id='+91 20 2542 4466' type='Fax' properties={}
id='Techmahindra.Com' type='Website' properties={}
id='Connect@Techmahindra.Com' type='Email' properties={}
id='Investor Grievance Redressal Policy' type='Policy' properties={}
-----
source=Node(id='Tech Mahindra Limited', type='Company', properties={}) target=Node(id='Sharda Centre, Off Karve Road, Pune - 411004, Maharashtra, India', type='Address', properties={}) type='LOCATED_AT' properties={}
source=Node(id='Tech Mahindra Limited', type='Company', properties={}) target=Node(id='+91 20 6601 8100', type='Phone', properties={}) type='HAS_PHONE' properties={}
source=Node(id='Tech Mahindra Limited', type='Company', properties={}) target=Node(id='+91 20 2542 4466', type='Fax', properties={}) type='HAS_FAX' properties={}
source=Node(id='Tech 

In [7]:
graph_documents

[GraphDocument(nodes=[Node(id='Tech Mahindra Limited', type='Company', properties={}), Node(id='Sharda Centre, Off Karve Road, Pune - 411004, Maharashtra, India', type='Address', properties={}), Node(id='+91 20 6601 8100', type='Phone', properties={}), Node(id='+91 20 2542 4466', type='Fax', properties={}), Node(id='Techmahindra.Com', type='Website', properties={}), Node(id='Connect@Techmahindra.Com', type='Email', properties={}), Node(id='Investor Grievance Redressal Policy', type='Policy', properties={})], relationships=[Relationship(source=Node(id='Tech Mahindra Limited', type='Company', properties={}), target=Node(id='Sharda Centre, Off Karve Road, Pune - 411004, Maharashtra, India', type='Address', properties={}), type='LOCATED_AT', properties={}), Relationship(source=Node(id='Tech Mahindra Limited', type='Company', properties={}), target=Node(id='+91 20 6601 8100', type='Phone', properties={}), type='HAS_PHONE', properties={}), Relationship(source=Node(id='Tech Mahindra Limited',

In [8]:
# Neo4j setup
os.environ["NEO4J_URI"]="neo4j+s://fd98554c.databases.neo4j.io:7687"
os.environ["NEO4J_USERNAME"]="neo4j"
os.environ["NEO4J_PASSWORD"]="xqABP_BqOm9OC0U6r-8-2OOd_jtpQSwGMy32vlHmvGM"
os.environ["AURA_INSTANCEID"]="fd98554c"
os.environ["AURA_INSTANCENAME"]="Instance01"

graph = Neo4jGraph()

In [9]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [10]:
def showGraph():
    driver=GraphDatabase.driver(
        uri=os.environ["NEO4J_URI"],
        auth=(os.environ["NEO4J_USERNAME"],os.environ["NEO4J_PASSWORD"]),)
    session = driver.session()
    widget = GraphWidget(graph = session.run("MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t").graph())
    widget.node_label_mapping = "id"
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [1]:
#%pip install langchain-nomic

In [13]:
#Ollama Embeddings
# embeddings = OllamaEmbeddings(
#     model="mxbai-embed-large",
# )
import os
# #Nomic Embedding 
# os.environ["NOMIC_API_KEY"] = "nk-2DfRhh0f-ybdSqzbYpNCm19NqPGhTlr0k6vo4CwB99E"
# from langchain_nomic import NomicEmbeddings

# embeddings = NomicEmbeddings(
#     model="nomic-embed-text-v1.5",
#     # dimensionality=256,
#     # Nomic's `nomic-embed-text-v1.5` model was [trained with Matryoshka learning](https://blog.nomic.ai/posts/nomic-embed-matryoshka)
#     # to enable variable-length embeddings with a single model.
#     # This means that you can specify the dimensionality of the embeddings at inference time.
#     # The model supports dimensionality from 64 to 768.
#     # inference_mode="remote",
#     # One of `remote`, `local` (Embed4All), or `dynamic` (automatic). Defaults to `remote`.
#     # api_key=... , # if using remote inference,
#     # device="cpu",
#     # The device to use for local embeddings. Choices include
#     # `cpu`, `gpu`, `nvidia`, `amd`, or a specific device name. See
#     # the docstring for `GPT4All.__init__` for more info. Typically
#     # defaults to CPU. Do not use on macOS.
# )

#OpenAI Embeddings
# os.environ["OPENAI_API_KEY"] = ""
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()


In [14]:
driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

In [29]:

class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            #"You are extracting organization and person entities from the text.",
            "You are expert in extracting all possible entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


entity_chain = llm.with_structured_output(Entities)

In [43]:
entity_chain.invoke("Give me the summary of all the Policies?")

Entities(names=['Policies'])

In [45]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [47]:
print(graph_retriever("Give me the summary of all the Tech Mahindra Policies"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 116} for query: "CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "


Tech Mahindra - RUNS -> Peak Turbocharge Program
Tech Mahindra - IS -> Agile
Tech Mahindra - IS -> Responsive
Tech Mahindra - IS_A -> Global Organization
Tech Mahindra - COMMITMENT_TO -> Esg Leadership
Tech Mahindra - BELIEVES_IN -> Flexible Work Culture
Tech Mahindra - SUPPORT -> Non-Profit Making Activities
Tech Mahindra - INVOLVES -> Project Fortius
Tech Mahindra - INVOLVES -> Strategic Solutioning And Transformation
Tech Mahindra - INVOLVES -> Advisor Relations
Tech Mahindra - INVOLVES -> Stakeholders
Tech Mahindra - COMMITMENT -> Environmental Compliance
Tech Mahindra - LAUNCHED -> Mobile Devices
Tech Mahindra - LAUNCHED -> Iot Devices
Tech Mahindra - LOCATED_AT -> Sharda Centre, Off Karve Road, Erandwane, Pune, Maharashtra, India 411004
Tech Mahindra - HAS_WEBSITE -> Www.Techmahindra.Com
Tech Mahindra - OVERSEES -> Board Of Directors
Tech Mahindra - FOLLOWS -> 3-Year Roadmap
Tech Mahindra - ENGAGES -> Private Equity
Tech Mahindra - PLANNED_FOR -> Fy2025
Tech Mahindra - IMPACTED_B

In [48]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    print(graph_data)
    print("=====================================")
    print(vector_data)

    return final_data

In [49]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)   

In [53]:
#chain.invoke(input="")
#chain.invoke(input="")
#chain.invoke(input="")
#chain.invoke(input="Give me full extended summary on Tech Mahindra's policies")
#chain.invoke(input="who should be contacted to raise a complaint?")
#chain.invoke(input="Give me some insights on TechM flywheel")
#chain.invoke(input="take a high level view and tell me how Tech Mahindra is better than other companies")
chain.invoke(input="How is Tech Mahindra committed employee wellness?")
# chain.invoke(input="")


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 116} for query: "CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Ne

Tech Mahindra - RUNS -> Peak Turbocharge Program
Tech Mahindra - IS -> Agile
Tech Mahindra - IS -> Responsive
Tech Mahindra - IS_A -> Global Organization
Tech Mahindra - COMMITMENT_TO -> Esg Leadership
Tech Mahindra - BELIEVES_IN -> Flexible Work Culture
Tech Mahindra - SUPPORT -> Non-Profit Making Activities
Tech Mahindra - INVOLVES -> Project Fortius
Tech Mahindra - INVOLVES -> Strategic Solutioning And Transformation
Tech Mahindra - INVOLVES -> Advisor Relations
Tech Mahindra - INVOLVES -> Stakeholders
Tech Mahindra - COMMITMENT -> Environmental Compliance
Tech Mahindra - LAUNCHED -> Mobile Devices
Tech Mahindra - LAUNCHED -> Iot Devices
Tech Mahindra - LOCATED_AT -> Sharda Centre, Off Karve Road, Erandwane, Pune, Maharashtra, India 411004
Tech Mahindra - HAS_WEBSITE -> Www.Techmahindra.Com
Tech Mahindra - OVERSEES -> Board Of Directors
Tech Mahindra - FOLLOWS -> 3-Year Roadmap
Tech Mahindra - ENGAGES -> Private Equity
Tech Mahindra - PLANNED_FOR -> Fy2025
Tech Mahindra - IMPACTED_B

'Tech Mahindra is committed to employee wellness by fostering a flexible work culture, promoting diversity and inclusion, and ensuring a better work and workplace experience. They focus on talent management and learning and development to equip employees for success in a changing landscape. Additionally, they prioritize data privacy and security, providing safeguards for employees while respecting their privacy and dignity.'